Code is taken from https://www.kaggle.com/code/gpreda/honey-bee-subspecies-classification/notebook with minor adjusments

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

# you need to compress assets/ into bees.tar.gz and upload it to your google drive
!tar -xvf /content/gdrive/MyDrive/bees/bees.tar.gz

Streaming output truncated to the last 5000 lines.
bee_imgs/046_000.png
bee_imgs/046_004.png
bee_imgs/046_006.png
bee_imgs/046_007.png
bee_imgs/046_008.png
bee_imgs/046_009.png
bee_imgs/046_010.png
bee_imgs/046_011.png
bee_imgs/046_012.png
bee_imgs/046_013.png
bee_imgs/046_016.png
bee_imgs/043_064.png
bee_imgs/043_055.png
bee_imgs/043_056.png
bee_imgs/043_057.png
bee_imgs/043_058.png
bee_imgs/043_059.png
bee_imgs/043_060.png
bee_imgs/043_061.png
bee_imgs/043_062.png
bee_imgs/043_063.png
bee_imgs/043_008.png
bee_imgs/043_010.png
bee_imgs/043_011.png
bee_imgs/043_012.png
bee_imgs/043_013.png
bee_imgs/043_014.png
bee_imgs/043_015.png
bee_imgs/043_016.png
bee_imgs/043_017.png
bee_imgs/043_018.png
bee_imgs/043_019.png
bee_imgs/043_020.png
bee_imgs/043_021.png
bee_imgs/043_022.png
bee_imgs/043_023.png
bee_imgs/043_024.png
bee_imgs/043_025.png
bee_imgs/043_026.png
bee_imgs/043_027.png
bee_imgs/043_028.png
bee_imgs/043_029.png
bee_imgs/043_030.png
bee_imgs/043_031.png
bee_imgs/043_032.png
bee_

In [3]:
import pandas as pd
import numpy as np
import skimage
import skimage.io
import skimage.transform
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import (
    Dense,
    Conv2D,
    Flatten,
    MaxPool2D,
)
from keras.preprocessing.image import ImageDataGenerator

In [4]:
IMAGE_PATH = "bee_imgs/"
IMAGE_WIDTH = 100
IMAGE_HEIGHT = 100
IMAGE_CHANNELS = 3
RANDOM_STATE = 2018
TEST_SIZE = 0.1
VAL_SIZE = 0.2
CONV_2D_DIM_1 = 16
CONV_2D_DIM_2 = 16
CONV_2D_DIM_3 = 32
CONV_2D_DIM_4 = 64
MAX_POOL_DIM = 2
KERNEL_SIZE = 3
BATCH_SIZE = 32
NO_EPOCHS_1 = 5
NO_EPOCHS_2 = 10
NO_EPOCHS_3 = 50
PATIENCE = 5
VERBOSE = 1
CLASSES_CHANGE = {1: 2, 2: 1}

In [5]:
honey_bee_df = pd.read_csv("bee_data.csv")
lst = [
    "Italian honey bee",
    "Russian honey bee",
    "Carniolan honey bee",
    "VSH Italian honey bee",
    "Western honey bee",
]
honey_bee_df = honey_bee_df[honey_bee_df.subspecies.isin(lst)]
honey_bee_df = honey_bee_df.drop(
    ["date", "time", "location", "zip code", "health", "pollen_carrying", "caste"],
    axis=1,
)

In [6]:
train_df, test_df = train_test_split(
    honey_bee_df, test_size=TEST_SIZE, random_state=RANDOM_STATE
)
train_df, val_df = train_test_split(
    train_df, test_size=VAL_SIZE, random_state=RANDOM_STATE
)

In [7]:
def read_image(file_name, train=True):
    image = skimage.io.imread(IMAGE_PATH + file_name if train else file_name)
    image = skimage.transform.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT), mode="reflect")
    return image[:, :, :IMAGE_CHANNELS]

In [8]:
def categories_encoder(dataset, var="subspecies"):
    X = np.stack(dataset["file"].apply(read_image))
    y = pd.get_dummies(dataset[var], drop_first=False)
    return X, y

In [9]:
X_train, y_train = categories_encoder(train_df)
X_val, y_val = categories_encoder(val_df)
X_test, y_test = categories_encoder(test_df)

In [10]:
def create_model():
    model = Sequential()
    model.add(
        Conv2D(
            CONV_2D_DIM_1,
            kernel_size=KERNEL_SIZE,
            input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS),
            activation="relu",
            padding="same",
        )
    )
    model.add(MaxPool2D(MAX_POOL_DIM))
    model.add(
        Conv2D(
            CONV_2D_DIM_2, kernel_size=KERNEL_SIZE, activation="relu", padding="same"
        )
    )
    model.add(Flatten())
    model.add(Dense(y_train.columns.size, activation="softmax"))
    model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
    )

    return model

In [11]:
image_generator = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=180,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
)

image_generator.fit(X_train)

In [12]:
model = create_model()
train_model1 = model.fit_generator(
    image_generator.flow(X_train, y_train, batch_size=BATCH_SIZE),
    epochs=NO_EPOCHS_1,
    validation_data=[X_val, y_val],
    steps_per_epoch=len(X_train) / BATCH_SIZE,
)

Epoch 1/5


<ipython-input-12-ab46211eefce>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  train_model1  = model.fit_generator(image_generator.flow(X_train, y_train, batch_size=BATCH_SIZE),


96/96 [==============================] - 13s 110ms/step - loss: 0.5467 - accuracy: 0.8436 - val_loss: 0.1742 - val_accuracy: 0.9532
Epoch 2/5
96/96 [==============================] - 11s 111ms/step - loss: 0.1868 - accuracy: 0.9343 - val_loss: 0.0866 - val_accuracy: 0.9688
Epoch 3/5
96/96 [==============================] - 12s 121ms/step - loss: 0.1153 - accuracy: 0.9607 - val_loss: 0.1776 - val_accuracy: 0.9467
Epoch 4/5
96/96 [==============================] - 8s 86ms/step - loss: 0.1047 - accuracy: 0.9668 - val_loss: 0.0891 - val_accuracy: 0.9662
Epoch 5/5
96/96 [==============================] - 10s 102ms/step - loss: 0.0972 - accuracy: 0.9678 - val_loss: 0.0620 - val_accuracy: 0.9844


In [39]:
import json
import numpy as np
import requests
from requests import Session
from PIL import Image
from io import BytesIO
import base64

url = "https://updateme.ngrok-free.app"


def read_base64_image(image_data, width, height):
    image = Image.open(BytesIO(base64.b64decode(image_data)))
    image = np.array(image)

    image = skimage.transform.resize(image, (height, width), mode="reflect")

    cropped_image = image[:height, :width, :]

    return cropped_image


subspecies = {
    1: "Italian honey bee",
    2: "Russian honey bee",
    0: "Carniolan honey bee",
    3: "VSH Italian honey bee",
    4: "Western honey bee",
}

# test_input = "iVBORw0KGgoAAAANSUhEUgAAACcAAAAsCAIAAABpBjahAAALU0lEQVRYCX3Ba3MV15UG4Het3Xt3n6uEDgJs7sjwJZVK8jfnw9TYDhk7f8gfxlW2ZN1MxeNgGxQDktC59W3v9U7TqZOCSnmeR45PngIws5SSmbVNcs7l+UBVRRx6JNEzM/REBBsigg+xJyIk0UspASCpPTk8+tzMSAIgaQkiEkIh7zh8iCR+A0kRISkiAGKMzjlskAQgPQBy8N2nJM1MRAAInIg451UVUPwGkuiJCEkAJNETEQAiwh7eIyLoyf7Bf3EDQOYK55yIABARbIgINkjiPexhQ0QAiAhJ/AY5+O5TbKgqmMkG3iMi2CCJnogAMDOS2JAeeiRFhCQ+JAfffaqqskFzIoL/F0n0RASAmZHEhojgPSJCEh+So+M/ywYAmgNAEj0RQU9EAJDEBkkRAWBmJAGYGQBVxXtEBP9GDr77FICqigg6zACICAARwQZJbJDEe0gCMDNsqCo2zExV8SH5n6//wzknIugJfJZlqioieI+ZkTQzvEdV8RvMDBuqig/JN9/+pyV0zKCqzvmOqooIYNgwMwAk8R7pEB0RQUciAOmpqoiAStLMSBoEgAk6cnj0uSWQTIkA8nzgnBMRvGPYUFX0RAQ9kgCEMDP8k0QAJAGYGTpUEQEgIlAHwAQdOTr+M0kzo0lRFM75lBJJACLEhqqiJyIJlI4RgCPMTGEk13W1XC5Xq0VKycxCCOPx1HfyPIRAqHMuKcxMjk+eiggAgcvzPCWmlEjiHcOGiOBfnJqZGAFcnV+0nbqMMV7Or8wspdZ6IlIUwyzLptvvDEcT59yyLkMIcnDwOcTYE5HMBeeciLCHD0kPAEnEFGO8+vVNjHG1XsQYm9SgIwbAzNgxMbPBaHzjxo3J1rUQwio2MUY5PHy6XM2bpkkpARgNJ+Px2DlH0szwIZIAQght21rTppTe/HJGMqbGzCIjOmIA2GvqaGaD0bgoivF0ezQaXa6XZVnK0dFfFsur6p21c24ymWxtbYVQtG1rZgBI4l+MAJxI27a2rheLhSQDoESnrNtOSqlpmsFgEGMEkGVZNKuqKhT51tZWsT3JskwOD5+W1aqu69VqQXI8Hm9vb3ufk0wpkTQzVUVPiHfMqqpq5qu6rtlGAEp0Lq8WTdPEXgihbVsA3vvxdNo0TSjycef6tTzP5fDwadNW656ZjUaj8XgcQkDHaGYkAZgZACEBqOnV1VU9XzVNo8nYA3B1tYgxmhkAkm3bmsB7f202U9VQ5N77nZuzLMvk8PBp01Z1Xc/nc+fcaDTK8zzLMuecxQSAJADrwQzA4nJRlmVa123bxrJCj6QZYozOOQAxxqZpTBBCmGxtee/zQeGcu3Zjx3sv+/ufmVmMkSR6WccJACcEQCR2TEjGOi6Xy/Ozi6ZpEGlmSEYypSQi3nsz896HEFTVe09xbdu6IhsMBgnJe789m3jv5dtvP2UvpQRAe07RcUIARGLHpGmaxdvFcrm8ej1v21ZNSDpIJ8syEQGQUlLVoihms5n3PlGapmGGPM+b1DjnptdGIQTZ3/+MGwBEBEDmRDswAMZIsqnjvPNmsVwumzKamUcGQERU1TknIiGEsizbtvXe37x50zlXt6mqqqQ2GAxMkvf+o49nIQTZ3/8MAHsApKdCVXVCEYmpIXn1djGfz69ez9frNaN0BlmhquzFGElOp9OmacxMREIIzrmybsuyXLfldDrNh2E4HD58dDuEIAcHn5MEQBKA9DJ10kHrnCM5n8/PX5cXFxfzy2XTNAL3jug/OedEBICIAHDOaa9pmrpuO84ryczrYDC4e+/mbDaT/f3PAJDEhqo6URFxmlS1qqo3b9788tNFWZaMYp2E1GmjmeV5HkLw3gMwMwBZlqkqyfiOdQajom3bdbkUkbv3bt67d0/29z8DwA0AqupdpqrO1SJyeV6enZ1dXFhHkANwkkjGGFNKMUbrhRBiHUXEOaeqAETERDt55uq6vpxfNk2z9/jukydPZH//M5IASJoZAFUNmVfVLGs6r/4xPzs7WyxcSqmumFLK1JxzqpplGQAzizECEBPnnKoCSCmJiAu5dmhlWS7LZdu2jz658+TJEzk4+JwkgNQD4JwLIWRZJlKllP7+v//44YcflgvvnNNszE5LM3OKPM8zb23bVlXVtu14OPTehxDquq6qyntfFEPv/Xq9XK1Wi8XVdDq9//CjBw8eyMnJF6lnZjFGks65PM+99yJVXdffnzw/PT1tm3FRFKHYNrNUp7ZtBZbnOVGTbNvWzIZFMRgMzKzuhRCGw3EIYbG4Wi6XTVPt7u7ef/jRvXv35OjoLwBSL8ZoZqo6GAy8987F9Xp9ePC34+PjVTmcTqfeT9lpQ13XiqiqkEZEfKad8Sh476tqXZYlSek4LyJtW6/X68GoePTo0b0Ht3Z2duT4+L9Jpl6M0cxUdTAY+HdstVp9f/L89PT05T/ara2tELbMrC21ruvUllmWTaa5qg4Hufd+UDjv/Xq9rKpKVVNKq7KOMQLmvb97/86jR49ufnRNOicnX5A0sxhj27YpJVUtisJ7n+cIITw7/emrr756cdao6ni8G0JYXcW6rmFNnufjSRgMBtPJYDqdts1CVY1N27bL5ZJkoqiqc7K7u3vr9s3r16+Pxt7M5PT0S5Jm1vZSSqpaFIX3vigky7Jfz66Ojo5Ovn8zn8+BwXQ6rZZsmkbYZlm2MxtvbW3d2L026BQSY1wsL1e90CmGqvrxx7e2t7cH4yKE4LIEQE5PvyRpZjHGpmlSSqpaFEWWZcFLlmWWspcvX54++/Wnn366eNPEGAETkcloMBqN7ty9MZlMrs+2BoMBLNZ1fXH56u3bt8653d3dWx/f8d6HkHUMREcaAHJy8gUAM4sbqhpC8N47Ne+9IK+q6vIKz58/Pz3+5cWLF2QaDocf3dzd2tp68PDjoiiuz7aKoqjWy9Vq9frN2cXFhareu3fvwaNPvPdtW1sHBGCsVFVOTv5KJpIxGsm2rbMsU9Usy0TEOed9LiI0t1gs/v78b8+ePVsul977nZ2d+/fv3759O++JyKuz87IsLy4u3r59C2Bvb+/mR7eyLEOPSADMTFXl+PhLwEimxE6MjXNOeq4XQiEiYJZSmi8uvv/+++fPn4cQbt26tbe3t7u7Kz0zu7pYVlV1fn7++vVrAI8fP969ecM5hx6RAJiZqsrx8ZdkAkAKgKapmqYpy1JVt7e38zz33jvnSJpZ27ZHR0fffvut935vb+/3v//9ZDLhRozW+fXXX3/++Wfn3N7e3mw2w4aZoaeqcnz8JWB4R0m2bT2fz9frtXNue3t7Mpk451Q1z3MzW6/Xh4eHX331VQjh8ePHf/jDH3Z2dtjDOwrg7Ozsxx9/zPP8k08+2draMjMAZoYNVZWjoy8AI2kG51zTVGVZLpdLEdnZ2RmNRiLinFPVEMJisfjmm2++/vrrpmn+9Kc//fGPf5zNZiklkgBIMbPLy8uXL1/meX7nzp3xeGw9fEgODp6SCQApqmoWy7KsqkpEJpPJeDxWVQAiEkJYrVZHR0fPnj0ry/LJkye/+93vrl+/3jRNSgmAGUiuVqtXr14BuHv37mg0ijECIImeiACQo6MvyCQigJJsmmq1WqWUQgh5nk+nUzMDkFLy3scYnz9//uOPP6rqgwcP7t+/PxwOm6ZJKQEgheRyuXz16hWABw8eFEURYwRAEj0RASAnJ38VYU9IVtV6uVySDCHkeT6dTlNKANhT1RcvXrx582Y0Gt2+fXs2mwFo25YkAFIAnJ+fv3z5Ms/zhw8f5nkeY8S/kZOTv5LEO0ayjXVZlm3beu+Hw6GIZFmmqiSXvdVqlWXZ7u7u9va2qgIwM5IABI5kXdfn5+eqOpvNvPckAZgZNlT1/wDE5OmFK4PW+gAAAABJRU5ErkJggg=="
# image = read_base64_image(test_input, IMAGE_WIDTH, IMAGE_HEIGHT)
# image = np.expand_dims(image, axis=0)

# probabilities = model.predict(image)
# predicted_class_index = np.argmax(probabilities)
# subspecie = subspecies[predicted_class_index]
# print(subspecie)

session = Session()

while True:
    submission_data = {}
    try:
        response = requests.get(f"{url}/images")
        image_data = json.loads(response.text)

        for item in image_data:
            for identifier, base64_data in item.items():
                bid = identifier
                image = read_base64_image(base64_data, IMAGE_WIDTH, IMAGE_HEIGHT)
                image = np.expand_dims(image, axis=0)

                probabilities = model.predict(image)
                predicted_class_index = np.argmax(probabilities)
                subspecies = subspecies[predicted_class_index]
                submission_data[bid] = subspecies

        print(submission_data)
        ans = session.post(f"{url}/submit", json=submission_data)
        print(ans.text)

        if "HCSC24" in ans.text:
            break  # Exit the loop if "HCSC24" is found in the response
    except Exception as e:
        print(f"An error occurred: {e}")

1/1 [==============================] - 0s 33ms/step
{'4c-teclS4auLu47bnO9U5qtnxq-NcvC2rNK_dbHRbCrhrNB8iC72yHx4TWw-ha7o0_mg': 'Carniolan honey bee', '9kCYQZaZwdYnpdA4hTtzkxZO35AJCRFwLIIfQe7h8kwm8D3PWWhjItdCFrMgTq6NiksM': 'Russian honey bee', 'zb5CgHIgU_7BIYGNwxciciW8ukbNZ9SwkVujb-Pgvgw_u-cv0Fkay3b9FPB70ONDuKmj': 'Italian honey bee', '1nvlqYhnIXF5dbQOy8u128HRBvQ55i8vAD_BN5cpjTX5Q4nVjMpNung8CMtg4oAM4ST1': 'Italian honey bee', 'fsWxBFtm99GdqOvcjTFG_14zLKHWDYScdkV6DkqxQ5l2dgIN3DGJGOM0kikYBinNu0qL': 'Italian honey bee', 'vlN0yLEJqivA9-Lh_yc5UmQLWERkWlb9q2r3ry_dGW-rtbBAgxdACOeDlhGRQi47WzV2': 'Italian honey bee', '9yokV1BCpJ42HxHHzPqPe1eiALAdCGQWnPWaKUmyrm2oYSSPpbDW2EeJ_DnJLojn2H0_': 'Italian honey bee', 'F41YM3AhLGMW3u-fX3tUVJGRNgVhUVG7_YGUs97EKsci48CdlDHFZPEjnMEkFtTPnWHT': 'Italian honey bee', 'gRzNKnC_hk5apqbncEaCB3WUD8uUCWP-w8hJvUKfMR0fcd2Tuo1AuC_yuBj5ZJLED4e1': 'Italian honey bee', 'Kbx6ob-iPM2-2F0Cz6a3Ndr8yAY1607suyc4GUfEsoGfrbs4kufOuOq8RlZbBTw8O4fO': 'Western honey bee', 'E12hvn4t4z3Pna